In [ ]:
!pip install -q langchain
!pip install -q ragas
!pip install -q faiss-cpu
!pip install -q langchain-openai
!pip install -q tiktoken

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["OPENAI_API_KEY"]= OPENAI_API_KEY

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [ ]:
# Load the data
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

In [ ]:
# Chunk the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(chunks, embeddings)

In [ ]:
retriever=vectordb.as_retriever()

In [ ]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d7a48168190>, search_kwargs={})

In [ ]:
## Defining LLM
llm = ChatOpenAI(temperature= 0.6)

In [ ]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("what did the President say about Justic Breyer?")

'The President honored Justice Stephen Breyer for his service and announced the nomination of Judge Ketanji Brown Jackson to continue his legacy.'

In [ ]:
questions=["what did the President say about Justic Breyer?",
           "What did the President say abou Intel's CEO?",
           "What did the President say about gun violence?"]

In [ ]:
ground_truths=["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.",
              "The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion.",
              "The president asked Congress to pass proven measures to reduce gun violence."]

In [ ]:
answers = []
contexts = []

In [ ]:
# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

In [ ]:
answers

["The President honored Justice Stephen Breyer for his dedication to serving the country and thanked him for his service. The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence.",
 "The President stated that Intel's CEO, Pat Gelsinger, is ready to increase their investment from $20 billion to $100 billion.",
 'The President called for Congress to pass measures to reduce gun violence, including universal background checks, banning assault weapons, and repealing the liability shield for gun manufacturers. He emphasized that these laws save lives and do not infringe on the Second Amendment.']

In [ ]:
contexts

[['Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.',
  'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Borde

In [ ]:
# To dict
data = {
    "question": questions,
    "ground_truth": ground_truths,
    "answer": answers,
    "contexts": contexts,
}

In [ ]:
from datasets import Dataset
dataset=Dataset.from_dict(data)

In [ ]:
dataset

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 3
})

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [ ]:
result= evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy],
    llm= llm,
    embeddings= embeddings)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 0.9444, 'context_recall': 1.0000, 'faithfulness': 0.7500, 'answer_relevancy': 0.9021}

In [ ]:
df = result.to_pandas()

In [ ]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy
0,what did the President say about Justic Breyer?,"[Tonight, I’d like to honor someone who has de...",The President honored Justice Stephen Breyer f...,The president said that Justice Breyer has ded...,1.000000,1.0,1.00,0.888097
1,What did the President say abou Intel's CEO?,[Up to eight state-of-the-art factories in one...,"The President stated that Intel's CEO, Pat Gel...",The president said that Pat Gelsinger is ready...,1.000000,1.0,0.50,0.910399
2,What did the President say about gun violence?,[And I ask Congress to pass proven measures to...,The President called for Congress to pass meas...,The president asked Congress to pass proven me...,0.833333,1.0,0.75,0.907931
